In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calc_lift(y_actual,y_prob,bins=10):
# """
# Takes input arrays and trained SkLearn Classifier and returns a Pandas
# DataFrame with the average lift generated by the model in each bin

# Parameters
# -------------------
# x:    Numpy array or Pandas Dataframe with shape = [n_samples, n_features]

# y:    A 1-d Numpy array or Pandas Series with shape = [n_samples]
# IMPORTANT: Code is only configured for binary target variable
# of 1 for success and 0 for failure

# clf:  A trained classifier algorithm
# bins: Number of equal sized buckets to divide observations across
#   Default value is 10
# """
    cols = ['actual','prob_positive']
    data = [y_actual,y_prob[:,1]]
    df = pd.DataFrame(dict(zip(cols,data)))
    df.sort_values(by='prob_positive', ascending=True, inplace=True)

    #Observations where y=1
    total_positive_n = df['prob_positive'].sum()
    #Total Observations
    total_n = df.index.size
    natural_positive_prob = total_positive_n/float(total_n)
    idx_positive_prob = 1

    #Create Bins where First Bin has Observations with the
    #Highest Predicted Probability that y = 1
    df['bin_positive'] = pd.qcut(df['prob_positive'],bins,labels=False)
    #Rearrange bins into decreasing order of probability
    df['bin_positive'] = 1 + max(df['bin_positive']) - df['bin_positive']
    
    pos_group_df = df.groupby('bin_positive')
    #Percentage of Observations in each Bin where y = 1 
    lift_positive = pos_group_df['actual'].sum()/pos_group_df['actual'].count()
    lift_index_positive = (lift_positive/natural_positive_prob)
    
    
    #Consolidate Results into Output Dataframe
    lift_df = pd.DataFrame({'baseline_percentage':natural_positive_prob
                            ,'lift_percentage':lift_positive
                            ,'baseline_index':idx_positive_prob
                            ,'lift_index':lift_index_positive
                           })
    
    return lift_df
	
def plot_lift_chart(lift: pd.DataFrame):
    plt.figure(figsize=(12,6))
    plt.plot(lift['baseline_index'], 'r-', label='Lift from random selection')
    plt.plot(lift['lift_index'], 'g*-', label='Lift from model selection')
    plt.legend(loc='best')
    plt.xticks(range(1, lift.index.values.max()+1))
    plt.show()
